# Check Payroll
Do each agency seperately. Then do statewide and citywide payroll because we need those numbers for pension and fringe calculations.

Important to include in methodolgoy: no way to know which payroll dollars are federal, they don't list source the same way. We could try and estimate or leave it be. Don't have a methodology to estimate but could use fraction of payroll expenditure record that is federal. 

## Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [3]:
sys.path.insert(0, os.path.abspath("../.."))

In [4]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40

## Get Agencies

In [20]:
from Initialize_Agencies import get_agencies

/opt/anaconda3/lib/python3.7/importlib/__init__.py:169: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  _bootstrap._exec(spec, module)


In [32]:
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/Agency_Classes_Big.py:90: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/Agency_Classes_Big.py:90: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()


2016   354100737.000
2017   372407936.000
2018   407607260.000
2019   422895141.000
Name: Total Expenditures, dtype: float64


## Save payroll to single dataframe

In [ ]:
payroll_df = pd.DataFrame(columns=yr)
for _, agency in agencies.items():
    payroll_df.loc[agency.official_name,:] = agency.payroll_by_year

In [9]:
payroll_df

,2016,2017,2018,2019
TRIAL COURT (TRC),488805222.036,490351865.595,505910980.367,521454178.758
COMMITTEE FOR PUBLIC COUNSEL SERVICES (CPC),207452633.424,222600323.454,222114813.876,231351800.938
DEPARTMENT OF CORRECTION (DOC),409921285.956,408114267.839,415989076.826,430854996.529
SUFFOLK DISTRICT ATTORNEY (SUF),15567170.580,15828355.790,16029452.963,17290861.498
SHERIFF DEPARTMENT SUFFOLK (SDS),81241417.138,82071839.532,84101778.220,85006459.731
DEPARTMENT OF STATE POLICE (POL),353785069.236,363805343.023,368611410.948,363778728.650
MASSACHUSETTS BAY TRANSPORTATION AUTHORITY (MBT),28140924.000,27914160.000,28731767.045,33395630.395
DISTRICT ATTORNEY ASSOCIATION (DAA),869460.027,876400.379,924391.941,947636.671
SHERIFFS DEPARTMENT ASSOCIATION (SDA),244370.730,234981.225,242014.830,222441.105
PAROLE BOARD (PAR),14595577.428,13987429.581,14239922.074,15335637.922


### Make custom function to find % of payroll dollars that are federal

In [10]:
def pcnt_payroll_expend_fed(expenditures):
    expenditures["budget_fiscal_year"] = expenditures["budget_fiscal_year"].astype(int)
    expenditures["amount"] = expenditures["amount"].astype(float)
    
    federal_expend = expenditures.query("appropriation_type == '(4FN) FEDERAL GRANTS'")
    federal_payroll_expend =  federal_expend[federal_expend["vendor"].str.contains("(?i)payroll")]
    federal_payroll_expend_by_year = federal_payroll_expend.groupby("budget_fiscal_year")["amount"].sum()

    payroll_expend = expenditures[expenditures["vendor"].str.contains("(?i)payroll")]
    payroll_expend_by_year = payroll_expend.groupby("budget_fiscal_year")["amount"].sum()
    
    return federal_payroll_expend_by_year/payroll_expend_by_year

## Trial Court (checks methodology for all state agencies except CPCS and MBTA)

In [11]:
trial_court_payroll = pd.DataFrame(client.get("rxhc-k6iz", where = "department_division = 'TRIAL COURT (TRC)'"\
                                              "AND Year >= 2018 AND Year <= 2019", limit =999999))

In [17]:
TRC_expenditures = pd.DataFrame(client.get("pegc-naaa", where = "department = 'TRIAL COURT (TRC)'"\
                                              "AND budget_fiscal_year = 2019", limit =999999))

In [18]:
trial_court_payroll.shape

(14876, 20)

In [19]:
trial_court_payroll["pay_total_actual"] = trial_court_payroll["pay_total_actual"].astype(float)

In [20]:
TRC_2018_payroll = trial_court_payroll[trial_court_payroll["year"] == "2018"]["pay_total_actual"].sum()
TRC_2019_payroll = trial_court_payroll[trial_court_payroll["year"] == "2019"]["pay_total_actual"].sum()

Check that fraction of payroll expenditures that are federal in 2019 is correct

In [21]:
fraction_non_federal = pcnt_payroll_expend_fed(TRC_expenditures)
fraction_non_federal.loc[2019]

0.0012947292015284225

In [22]:
TRC = agencies["trial_court"]

In [23]:
assert np.isclose(fraction_non_federal.loc[2019],
        TRC.fraction_payroll_federal.loc[2019])

In [24]:
assert np.isclose(np.mean([TRC_2018_payroll, TRC_2019_payroll])*(1-fraction_non_federal.loc[2019]), 
                  payroll_df.loc["TRIAL COURT (TRC)", 2019])

## CPCS

For CPCS, add R24

In [40]:
CPCS_payroll = pd.DataFrame(client.get("rxhc-k6iz", where = "department_division = 'COMMITTEE FOR PUBLIC COUNSEL SERVICES (CPC)' AND "\
                                                            "Year >= 2016 AND YEAR <=2017", limit =999999))

In [41]:
CPCS_expenditures = pd.DataFrame(client.get("pegc-naaa", where = "department = 'COMMITTEE FOR PUBLIC COUNSEL SERVICES (CPC)' AND "\
                                                            "budget_fiscal_year = 2017", limit =999999))

In [42]:
CPCS_payroll["pay_total_actual"] = CPCS_payroll["pay_total_actual"].astype(float)
CPCS_expenditures["amount"] = CPCS_expenditures["amount"].astype(float)

In [43]:
fraction_CPCS_federal = pcnt_payroll_expend_fed(CPCS_expenditures)

In [44]:
CPCS = agencies["CPCS"]

In [45]:
CPCS_expenditures= CPCS_expenditures[((CPCS_expenditures["appropriation_type"] != "(4FN) FEDERAL GRANTS"))]

In [46]:
CPCS_2016_payroll = CPCS_payroll[CPCS_payroll["year"]=="2016"]["pay_total_actual"].sum()
CPCS_2017_payroll = CPCS_payroll[CPCS_payroll["year"]=="2017"]["pay_total_actual"].sum()

In [47]:
public_counsel_services = CPCS_expenditures[CPCS_expenditures["object_code"].str.contains("R24")]

In [48]:
CPCS_2017_payroll_redone = np.mean([CPCS_2016_payroll, CPCS_2017_payroll])*(1-fraction_CPCS_federal.loc[2017])\
+ public_counsel_services["amount"].sum()

In [54]:
assert np.isclose(CPCS_2017_payroll_redone, 
           CPCS.payroll_by_year.loc[2017])

Ok great. Want to be absolutely sure that R24 is being handled the right way. If R24 comes through payroll could it be counted twice? Possibly yes. So add code to RR_correction that removes payroll dollars from our public counsel df. Shouldn't effect final results

In [ ]:
CPCS = agencies["CPCS"]

In [ ]:
CPCS.R24_by_year

In [ ]:
CPCS.payroll_by_year

In [ ]:
payroll_df.loc["COMMITTEE FOR PUBLIC COUNSEL SERVICES (CPC)"]

## MBTA

MBTA payroll is more complex. Payroll record starts in 2017. For years 2015 and 2017 there are pdf's of earnings posted on the MBTA's website that are used as a stand-in. So the data available is:
 - 2015: pdf
 - 2017: pdf and cthru (pdf is used)
 - 2018: cthru
 - 2019: cthru
Usually fiscal year 2016 would be an average of 2015 and 2016. Because there is no 2016 data, only 2015 data is used. Likewise for 2017, only 2017 data is used. 2018 and 2019 are averages of 2017/2018 and 2018/2019

First thing to check: does pdf scanner work correctly? To answer this, put in print statements. Did that, all seems gravy. To re-check all I need to do is move .json files with scraped results to another folder and the print statements tell me if it's working

In [55]:
MBTA_payroll = pd.DataFrame(client.get("rxhc-k6iz", where = "department_division = 'Massachusetts Bay Transportation Authority (MBT)' AND "\
                                                            "Year >= 2018 AND YEAR <=2019", limit =999999))

In [56]:
MBTA_payroll["pay_total_actual"] = MBTA_payroll["pay_total_actual"].astype(float)
MBTA_cop_payroll = MBTA_payroll[(MBTA_payroll["position_title"].str.contains("(?i)police")) |
                                (MBTA_payroll["position_title"].str.contains("(?i)sergeant"))]
MBTA_cop_payroll_2018 = MBTA_cop_payroll[MBTA_cop_payroll["year"]=="2018"]["pay_total_actual"].sum()
MBTA_cop_payroll_2019 = MBTA_cop_payroll[MBTA_cop_payroll["year"]=="2019"]["pay_total_actual"].sum()

In [57]:
assert(np.isclose(np.mean([MBTA_cop_payroll_2018, MBTA_cop_payroll_2019]), payroll_df.loc["MASSACHUSETTS BAY TRANSPORTATION AUTHORITY (MBT)", 2019]))

I checked the scraper by hand on Monday Dec 20

## Statewide Payroll

In [22]:
sys.path.insert(0, os.path.abspath("../../fringe"))
from Statewide_Fringe import Total_Statewide_Payroll

In [23]:
total_statewide_payroll = Total_Statewide_Payroll(client)

In [25]:
total_payroll_FY_2018 = pd.DataFrame(client.get("rxhc-k6iz", where = "Year >= 2017 AND YEAR <=2018", limit =999999))

In [27]:
total_payroll_FY_2018["pay_total_actual"] = total_payroll_FY_2018["pay_total_actual"].astype(float)

In [32]:
total_payroll_2017 = total_payroll_FY_2018[total_payroll_FY_2018["year"]=="2017"]["pay_total_actual"].sum()
total_payroll_2018 = total_payroll_FY_2018[total_payroll_FY_2018["year"]=="2018"]["pay_total_actual"].sum()

In [33]:
assert(np.isclose(np.mean([total_payroll_2017, total_payroll_2018]), total_statewide_payroll.loc[2018]))

Ok great that should have been easy lol

## Boston

Cop Pay: get earnings dataset for each year, break it down by title and whatnot to make sure it all looks good

Re-Download FY 2018 earnings dataset

In [5]:
boston_2018_earnings = pd.read_csv("Boston_2018_Earnings_forCheck.csv", engine="python")
boston_2019_earnings = pd.read_csv("Boston_Earnings_2019.csv", engine="python")

In [6]:
boston_2018_earnings = boston_2018_earnings[boston_2018_earnings["TOTAL EARNINGS"] != "TOTAL EARNINGS"]

In [7]:
boston_2018_earnings["TOTAL EARNINGS"] = boston_2018_earnings["TOTAL EARNINGS"].str.replace(",","").astype(float)
boston_2019_earnings["TOTAL EARNINGS"] = boston_2019_earnings["TOTAL EARNINGS"].str.replace(",","").astype(float)

In [8]:
police_earnings_2018 = boston_2018_earnings[boston_2018_earnings["DEPARTMENT_NAME"]=="Boston Police Department"]
police_earnings_2019 = boston_2019_earnings[boston_2019_earnings["DEPARTMENT_NAME"]=="Boston Police Department"]

In [ ]:
bostonPD = agencies["Boston PD"]

In [15]:
assert(np.isclose(np.mean([police_earnings_2018["TOTAL EARNINGS"].sum(),police_earnings_2019["TOTAL EARNINGS"].sum()]), bostonPD.payroll_by_year.loc[2019]))

In [17]:
sys.path.insert(0, os.path.abspath("../../pensions"))
from LocalPD_True_Payroll import remove_schools, True_Earnings

Check that code produces correct thing


In [18]:
total_earnings, PD_fraction_non_teacher, PD_fraction_total, PD_payroll, raw_data = True_Earnings("Boston PD")

In [27]:
Boston_PD_total_earnings, Boston_no_teachers, Boston_total_earnings = raw_data

In [28]:
PD_total_earnings.groupby("TITLE").size().sort_values(ascending=False)

TITLE
Police Officer                    6563
Police Detective                  1519
School Traffic Supv                944
Police Sergeant                    724
Police Sergeant (Det)              596
                                  ... 
Executive Coordinator                1
Technology Support Specialist        1
Director of Projects & Initiat       1
Captain/Academy Instructor           1
Police Offc RadioTech                1
Length: 199, dtype: int64

In [37]:
Boston_no_teachers.groupby("TITLE").size().sort_values(ascending=False)

TITLE
Police Officer                    6563
Paraprofessional                  6354
Fire Fighter                      4059
Lunch Hour Monitors               1840
Part-Time Cafeteria Attendant     1780
                                  ... 
Sr Staff Attorney (DND)              1
Construction Supervisor              1
Mngr of Exhibitations&Programm       1
Mgr of Rare Books&Manuscripts        1
Prin Clerk Typist 31                 1
Length: 1753, dtype: int64

In [36]:
Boston_total_earnings.groupby("TITLE").size().sort_values(ascending=False)

TITLE
Teacher                           26971
Police Officer                     6563
Paraprofessional                   6354
Fire Fighter                       4059
Substitute Teacher                 2779
                                  ...  
Street Lighting Compliance Spc        1
Grants Administrator                  1
Chief Officer (Acad)                  1
Chief Operating Officer               1
Mgr of Rare Books&Manuscripts         1
Length: 1771, dtype: int64

Ok that data is all quite clean. Next thing to check is make sure that I get non-teacher payroll at least mostly correct. It's possible I count some teachers towards what is supposed to be non-teacher citywide payroll because there are so many employers. This would result in a minor undercount of police pensions. 


In [105]:
boston_2018_earnings = boston_2018_earnings.rename(columns={"DEPARTMENT_NAME": "department"})
boston_2019_earnings = boston_2019_earnings.rename(columns={"DEPARTMENT_NAME": "department"})


Eyeball test says that all looks good

### Remove Federal money
We have total federal expenditures which is cool

In [33]:
bostonPD = agencies["Boston PD"]

In [34]:
bostonPD.federal_expenditures_by_year

2016    4349426
2017    2689144
2018    1024447
2019    1386617
Name: Total Federal/Private Grant Expenditures, dtype: object

In [35]:
bostonPD.fraction_spending_federal

2016   0.012
2017   0.007
2018   0.003
2019   0.003
dtype: object

In [36]:
bostonPD.budget_summary

,2016,2017,2018,2019,2020,2021
Payroll Budget,297299985.000,325090384.000,340550136.677,364655518.158,376094339.676,378944425.091
OT Budget,35619489.000,54966556.000,57076768.867,60115115.432,61415244.222,61500643.365
Total Budget,334810043.000,366418564.000,380087096.177,407114254.631,420864667.757,421589850.442
Payroll Appropriation,301839026.000,331053719.245,353648386.370,366234564.173,376709076.481,0.000
OT Appropriation,37966592.000,55969576.695,59235426.538,60319549.828,62014215.824,0.000
Total Appropriation,339300263.000,370044034.075,397129439.648,408288547.121,422530491.594,0.000
Payroll Expenditures,322000476.071,335073541.833,360633372.807,374141609.986,0.000,0.000
OT Expenditures,57790620.075,60618685.866,68313585.252,70204917.968,0.000,0.000
Total Expenditures,354100737.000,372407936.000,407607260.000,422895141.000,0.000,0.000


In [37]:
fraction_spending_federal_2018_handcalc = bostonPD.federal_expenditures_by_year.loc[2018]/\
bostonPD.budget_summary.loc["Total Expenditures", 2018]
fraction_spending_federal_2018_handcalc

0.002513318825577346

In [39]:
bostonPD.payroll_by_year

2016   368776534.543
2017   387116737.747
2018   402964978.537
2019   414839624.738
dtype: object

## Chelsea

One thing to double check: we don't have citywide non-teacher fraction for 2016 as we don't have citywide earnings that year. So we assume that change from year to year is linear and take the difference betweeen 2018 and 2017 and subtract it from 2017. Chose simplicity over something arguably more correct as it's not signficiant to final results. 
Do have the numbers ready so we can say how little it changes 

In [244]:
yr = list(range(2016,2020))
agencies = get_agencies(yr)

in add true earnings for  Boston PD
PD fraction non-teacher is 


2016   0.296
2017   0.309
2018   0.320
2019   0.330
dtype: float64

PD fraction overall is


2016   0.212
2017   0.222
2018   0.230
2019   0.237
dtype: float64


in add true earnings for  Chelsea PD
PD fraction non-teacher is 


fiscalyear
2017   0.224
2018   0.227
2019   0.247
2016   0.221
Name: totalpay, dtype: float64

PD fraction overall is


fiscalyear
2017   0.143
2018   0.145
2019   0.158
2016   0.140
Name: totalpay, dtype: float64

In [245]:
chelsea_2018_police["totalpay"].sum()/chelsea_2018_earnings["totalpay"].sum()

0.14495851262100326

In [232]:
chelsea_2018_earnings = pd.read_csv("Chelsea_Payroll_2018_forCheck.csv")

First step: make sure we are getting cops correctly. Data seems clean

In [147]:
chelsea_2018_police = chelsea_2018_earnings[chelsea_2018_earnings["department"]=="POLICE DEPARTMENT"]


In [225]:
chelseaPD = agencies["Chelsea PD"]
chelseaPD.payroll_by_year

fiscalyear
2017   12165124.100
2018   12770764.810
2019   14734808.950
2016    9685922.000
Name: totalpay, dtype: float64

Second step: develop a way to pull out MTRS employees correctly 

In [238]:
chelsea_2018_no_school, chelsea_2018_school = remove_schools(chelsea_2018_earnings, "Chelsea", "position")


Ok that all passes eyeball test. Last thing to check is whether 2016 number matches. Which it does.

Check that my code produces the correct thing

In [44]:
total_earnings, PD_fraction_non_teacher, PD_fraction_total, PD_payroll, raw_data = True_Earnings("Chelsea PD")

In [51]:
pd.set_option("display.max_rows",20)
Chelsea_PD_total_earnings, Chelsea_no_teachers, Chelsea_total_earnings = raw_data

In [52]:
Chelsea_PD_total_earnings.groupby("position").size().sort_values(ascending=False)

position
PATROLMAN                         27699
SERGEANT                           5495
LIEUTENANT POLICE DEPARTMENT       3017
CAPTAIN POLICE DEPARTMENT          1607
DETAIL RET POLICE PATROLMAN         692
                                  ...  
COMMUNITY YOUTH & FAMILIES ENG       77
DOMESTIC VIOLENCE ADVOCATE           75
RETIRED DETAIL POLICE CAPTAIN        47
CRIME ANALYST                        33
STUDENT OFFICER                       5
Length: 22, dtype: int64

In [57]:
Chelsea_no_teachers[Chelsea_no_teachers["position"].str.contains("(?i)teacher")]["department"]

22       EARLY LEARNING CENTER
28                  SYSTEMWIDE
110      EARLY LEARNING CENTER
120      EARLY LEARNING CENTER
127                 SYSTEMWIDE
                 ...          
84797    EARLY LEARNING CENTER
84863    EARLY LEARNING CENTER
84872    EARLY LEARNING CENTER
84895    EARLY LEARNING CENTER
85022    EARLY LEARNING CENTER
Name: department, Length: 6340, dtype: object

In [54]:
Chelsea_total_earnings.groupby("position").size().sort_values(ascending=False)

position
TEACHER                          48783
PARAPROFESSIONAL- TEACH AIDES    29714
PATROLMAN                        27699
FIREFIGHTER                      17330
BUILDING MAINTENANCE MAN          7741
                                 ...  
COMMUNITY SCHOOLS                    6
STUDENT OFFICER                      5
WEEKEND SUPERVISOR                   2
TEMP WIRE INSPECTOR                  2
BAKING INSTRUCTOR                    1
Length: 261, dtype: int64

## Revere

Double check cop, citywide earnings against budget pdf

**Important to say in methodology: Revere has seperate systems for pensions and healthcare for teachers**

In [7]:
reverePD = agencies["Revere PD"]

In [8]:
reverePD.payroll_by_year

2016   9047062.000
2017   9083601.000
2018   9357467.000
2019   9735336.000
Name: Payroll Adopted, dtype: float64

In [10]:
reverePD.PD_fraction

2016   0.333
2017   0.337
2018   0.335
2019   0.349
dtype: object

In [18]:
total_non_teacher_payroll_2016 = 27190424
total_non_teacher_payroll_2017 = 26989489

total_non_teacher_payroll_2018 = 27909365 
total_non_teacher_payroll_2019 = total_non_teacher_payroll_2018

In [24]:
assert np.isclose(reverePD.payroll_by_year.loc[2016]/total_non_teacher_payroll_2016, reverePD.PD_fraction.loc[2016])
assert np.isclose(reverePD.payroll_by_year.loc[2017]/total_non_teacher_payroll_2017, reverePD.PD_fraction.loc[2017])
assert np.isclose(reverePD.payroll_by_year.loc[2018]/total_non_teacher_payroll_2018, reverePD.PD_fraction.loc[2018])
assert np.isclose(reverePD.payroll_by_year.loc[2019]/total_non_teacher_payroll_2019, reverePD.PD_fraction.loc[2019])

## Winthrop

Double check cop earnings, we get citywide fraction from Revere

In [25]:
winthropPD = agencies["Winthrop PD"]
winthropPD.payroll_by_year

2016   2842909.000
2017   2927191.000
2018   3066911.000
2019   3803402.000
Name: Payroll Expenditures, dtype: float64

Important for refactor: 2016, 2017 numbers are on page 67 of FY 19 budget.  2018, 2019 are on page 73 of FY 21 budget